## Assignment B1

Predict the price of the Uber ride from a given pickup point to the agreed drop-off 
location. Perform following tasks: 
1. Pre-process the dataset. 
2. Identify outliers. 
3. Check the correlation. 
4. Implement linear regression and random forest regression models. 
Evaluate the models and compare the irrespective scores like R2, RMSE, etc.  

Step 1: Preprocessing the dataset

In [229]:
import pandas as pd

In [230]:
df = pd.read_csv('uber.csv')
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [231]:
df.describe()

,Unnamed: 0,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+05,200000.000000,200000.000000,200000.000000,199999.000000,199999.000000,200000.000000
mean,2.771250e+07,11.359955,-72.527638,39.935885,-72.525292,39.923890,1.684535
std,1.601382e+07,9.901776,11.437787,7.720539,13.117408,6.794829,1.385997
min,1.000000e+00,-52.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000
25%,1.382535e+07,6.000000,-73.992065,40.734796,-73.991407,40.733823,1.000000
50%,2.774550e+07,8.500000,-73.981823,40.752592,-73.980093,40.753042,1.000000
75%,4.155530e+07,12.500000,-73.967154,40.767158,-73.963658,40.768001,2.000000
max,5.542357e+07,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000


In [232]:
df.dtypes

Unnamed: 0             int64
key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [233]:
df = df.drop(['Unnamed: 0', 'key'], axis=1)
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [234]:
df.isna().sum()

fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [235]:
# handle null values in dropoff_longitude and dropoff_latitude
df.fillna(df['dropoff_longitude'].mean(), inplace=True)
df.fillna(df['dropoff_latitude'].mean(), inplace=True)
df.isna().sum()


fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [236]:
# Handle negative fareamount
df = df[df['fare_amount'] >= 0]

In [237]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,199983.000000,199983.000000,199983.000000,199983.000000,199983.000000,199983.000000
mean,11.362365,-72.527515,39.935816,-72.525538,39.923462,1.684493
std,9.897119,11.438266,7.720864,13.116923,6.799162,1.385979
min,0.000000,-1340.648410,-74.015515,-3356.666300,-881.985513,0.000000
25%,6.000000,-73.992065,40.734796,-73.991407,40.733823,1.000000
50%,8.500000,-73.981823,40.752592,-73.980092,40.753042,1.000000
75%,12.500000,-73.967154,40.767158,-73.963658,40.768001,2.000000
max,499.000000,57.418457,1644.421482,1153.572603,872.697628,208.000000


In [238]:
df.corr(method="pearson", numeric_only=True)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
fare_amount,1.000000,0.010432,-0.008460,0.009145,-0.011281,0.010403
pickup_longitude,0.010432,1.000000,-0.816461,0.833093,-0.845811,-0.000411
pickup_latitude,-0.008460,-0.816461,1.000000,-0.774849,0.701940,-0.001563
dropoff_longitude,0.009145,0.833093,-0.774849,1.000000,-0.916370,-0.000031
dropoff_latitude,-0.011281,-0.845811,0.701940,-0.916370,1.000000,-0.000490
passenger_count,0.010403,-0.000411,-0.001563,-0.000031,-0.000490,1.000000


In [239]:
# remove outliers from pickup/dropoff locations
def remove_outliers(feature):
    global df
    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]
    
remove_outliers("pickup_latitude")
remove_outliers("pickup_longitude")
remove_outliers("dropoff_latitude")
remove_outliers("dropoff_longitude")

df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,171169.000000,171169.000000,171169.000000,171169.000000,171169.000000,171169.000000
mean,9.012359,-73.981929,40.752519,-73.980935,40.752778,1.683623
std,4.939390,0.016110,0.021653,0.016904,0.022960,1.399014
min,0.000000,-74.026353,40.686252,-74.027438,40.687874,0.000000
25%,5.700000,-73.992917,40.737725,-73.992299,40.737588,1.000000
50%,8.000000,-73.982912,40.753372,-73.982013,40.753978,1.000000
75%,11.000000,-73.971443,40.766691,-73.970250,40.767489,2.000000
max,499.000000,-73.935240,40.815686,-73.932840,40.816622,208.000000


In [240]:
import math

def euclidean_distance(lat1, lon1, lat2, lon2):
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    distance = math.sqrt(dlat**2 + dlon**2)
    return distance

df['distance'] = df.apply(
    lambda row: euclidean_distance(
        row['pickup_latitude'], 
        row['pickup_longitude'], 
        row['dropoff_latitude'], 
        row['dropoff_longitude']
    ), axis = 1
)

In [241]:
df.drop(['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'], axis=1, inplace=True)

In [242]:
df.head()

,fare_amount,pickup_datetime,passenger_count,distance
0,7.5,2015-05-07 19:52:06 UTC,1,0.015140
1,7.7,2009-07-17 20:04:56 UTC,1,0.022103
2,12.9,2009-08-24 21:45:00 UTC,1,0.053109
3,5.3,2009-06-26 08:22:21 UTC,3,0.016528
5,4.9,2011-02-12 02:27:09 UTC,1,0.000000


In [243]:
# transform pickup_datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True)
df['hour'] = df['pickup_datetime'].dt.hour 

df.drop(['pickup_datetime'], axis=1, inplace=True)


In [244]:
df.head()

,fare_amount,passenger_count,distance,hour
0,7.5,1,0.015140,19
1,7.7,1,0.022103,20
2,12.9,1,0.053109,21
3,5.3,3,0.016528,8
5,4.9,1,0.000000,2


In [245]:
df.corr(method="pearson", numeric_only=True)

,fare_amount,passenger_count,distance,hour
fare_amount,1.000000,0.011885,0.703579,-0.000564
passenger_count,0.011885,1.000000,0.002218,0.013362
distance,0.703579,0.002218,1.000000,-0.018859
hour,-0.000564,0.013362,-0.018859,1.000000


In [246]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# standardize the splits
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [247]:
from sklearn.linear_model import LinearRegression

# linear regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

In [248]:
from sklearn.metrics import r2_score, root_mean_squared_error

print("R2 score: ", r2_score(y_test, y_pred))
print("RMSE: ", root_mean_squared_error(y_test, y_pred))

R2 score:  0.5529817837276233
RMSE:  3.1597281730286197


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# random forest regression model
# takes more time to train (comeback after 2 mins)
model = RandomForestRegressor(
    n_estimators=10,
    max_depth=5
)
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

In [250]:
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

print("R2 score: ", r2_score(y_test, y_pred))
print("RMSE: ", root_mean_squared_error(y_test, y_pred))

R2 score:  0.4865130181045111
RMSE:  3.38650585412746
